<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import hashlib
import requests
import sqlite3
import pandas as pd
from google.colab import userdata

PUBLIC_KEY = userdata.get('PUBLIC_KEY')
PRIVATE_KEY = userdata.get('PRIVATE_KEY')

# Criando o hash MD5
hashtre = hashlib.md5(str.encode(str(1) + str(PRIVATE_KEY) + str(PUBLIC_KEY)))

print(hashtre.hexdigest())
# Base URL da API da Marvel
base = "https://gateway.marvel.com"
url_montada_characters = f"{base}/v1/public/characters?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"

# Fazendo a requisição GET para a API e obtendo os dados
#response = requests.get('https://gateway.marvel.com/v1/public/characters', verify=False)
response = requests.get(url_montada_characters)

# Convertendo a resposta para JSON
dados = response.json()

# Acessando os resultados dos personagens
descricao = dados["data"]["results"]

# Conectando ao banco de dados SQLite (cria o banco se não existir)
conn = sqlite3.connect('marvel_characters.db')
cursor = conn.cursor()

# Criando a tabela para armazenar personagens
cursor.execute('''CREATE TABLE IF NOT EXISTS personagens (
                    id INTEGER PRIMARY KEY,
                    name TEXT,
                    description TEXT)''')

# Criando a tabela para armazenar quadrinhos
cursor.execute('''CREATE TABLE IF NOT EXISTS comics (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    character_id INTEGER,
                    comic_name TEXT,
                    FOREIGN KEY (character_id) REFERENCES personagens (id))''')

# Inserindo os dados dos personagens e quadrinhos no banco de dados
for char in descricao:
    char_id = char["id"]
    char_name = char["name"]
    char_description = char["description"]

    # Inserir personagem na tabela "personagens"
    cursor.execute('''INSERT OR IGNORE INTO personagens (id, name, description)
                      VALUES (?, ?, ?)''', (char_id, char_name, char_description))

    # Inserir quadrinhos associados ao personagem
    for comic in char["comics"]["items"]:
        comic_name = comic["name"]
        cursor.execute('''INSERT INTO comics (character_id, comic_name)
                          VALUES (?, ?)''', (char_id, comic_name))

# Commit e fechar a conexão
conn.commit()

# Lendo as tabelas com pandas
personagens_df = pd.read_sql_query("SELECT * FROM personagens", conn)
comics_df = pd.read_sql_query("SELECT * FROM comics", conn)

# Fechar a conexão
conn.close()

# Exibindo as tabelas
print("Tabela de Personagens:")
print(personagens_df)  # Exibe as primeiras linhas da tabela de personagens

print("\nTabela de Quadrinhos:")
print(comics_df)  # Exibe as primeiras linhas da tabela de quadrinhos


8d2db3e2472d0e9c304261ddbeda3584


SSLError: HTTPSConnectionPool(host='gateway.marvel.com', port=443): Max retries exceeded with url: /v1/public/characters?ts=1&apikey=02d1a63fd801bbc139518d3b14b920c1&hash=8d2db3e2472d0e9c304261ddbeda3584 (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1006)')))